In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
datapath = '/home/hky/Data/chenxu/MCmodelBany4.npz'

data = np.load(datapath)
for i in data.keys():
    print(i)

pri_e_num
pri_id
pri_e
pri_theta
pri_phi
pri_ne
pri_core_x
pri_core_y
pri_sump
pri_n_hit
Tibetevent
Tibet
MDevent
MD


In [3]:
pri_e_num = data['pri_e_num']
pri_id = data['pri_id']
pri_e = data['pri_e']
pri_theta = data['pri_theta']
pri_phi = data['pri_phi']
pri_ne = data['pri_ne']
pri_core_x = data['pri_core_x']
pri_core_y = data['pri_core_y']
pri_sump = data['pri_sump']
pri_n_hit = data['pri_n_hit']
Tibetevent = data['Tibetevent']
Tibet = data['Tibet']
MDevent = data['MDevent']
MD = data['MD']

In [4]:
calibfile = "/home/hky/for_mengy3/src16_1.8/userfile/mccaliblmzhai_20190312.cal"
detid = list()
x = list()
y = list()
z = list()

with open(calibfile, "r") as f:
    for line in f:
        detid.append(int(line.split()[0]))
        x.append(float(line.split()[1]))
        y.append(float(line.split()[2]))
        z.append(float(line.split()[3]))
detid = np.array(detid)
x = np.array(x)
y = np.array(y)
z = np.array(z)

In [5]:
ind = int(np.argpartition(pri_n_hit, -1)[-1:])

In [6]:
goodevent = list()
count = 0
for i in range(pri_id.shape[0]):
    if (-50 < pri_core_x[i] < 50) and (-50 < pri_core_y[i] < 50) and (pri_n_hit[i] > 300):
        goodevent.append(i)
        count += 1
        # if count == 100:
        #     break

In [7]:
len(goodevent)

769

In [8]:
for ind in goodevent:
    plt.figure(figsize=(40, 16))
    plt.subplot(1,2,1)
    Tibet_location_x = x[:996]
    Tibet_location_y = y[:996]
    Tibet_c = np.zeros(996)
    Tibet_s = np.ones(996)
    time = list()
    for i in Tibet[int(Tibetevent[ind]):int(Tibetevent[ind + 1]), :]:
        if i[0] == ind:
            if (i[3] <= 300) and (i[3] >= -300) and (i[2] > 0.6):
                Tibet_c[int(i[1])] = i[3]
                Tibet_s[int(i[1])] += np.log(i[2] / 0.6)
                time.append(i[3])
    Tibet_location_x = np.concatenate([[0], x[:996][np.where(x[:996] != 0)]])
    Tibet_location_y = np.concatenate([[0], y[:996][np.where(x[:996] != 0)]])
    Tibet_c = np.concatenate([[Tibet_c[0]], Tibet_c[np.where(x[:996] != 0)]])
    Tibet_s = np.concatenate([[Tibet_s[0]], Tibet_s[np.where(x[:996] != 0)]])

    cmap = cm.YlOrRd
    plt.scatter(Tibet_location_x[np.where(Tibet_c != 0)],
                Tibet_location_y[np.where(Tibet_c != 0)],
                s=Tibet_s[np.where(Tibet_c != 0)] * 50,
                c=Tibet_c[np.where(Tibet_c != 0)],
                cmap=cmap,
                # alpha=4 / 5,
                )
                
    plt.clim(-300, 300)
    plt.colorbar()
    plt.scatter(Tibet_location_x[np.where(Tibet_c == 0)],
                Tibet_location_y[np.where(Tibet_c == 0)],
                s=10,
                c='g',
                # alpha=4 / 5,
                )
    plt.scatter(pri_core_x[ind], pri_core_y[ind], marker='X', s=100, linewidths=1)
    arrowlength = 60
    plt.arrow(pri_core_x[ind] - arrowlength * np.cos(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              pri_core_y[ind] - arrowlength * np.sin(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              (arrowlength - 5) * np.cos(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              (arrowlength - 5) * np.sin(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              head_width=3,
              fc='b',
              ec='b')

    plt.xlim(-150,150)
    plt.ylim(-150,150)
    plt.xlabel('x/m')
    plt.ylabel('y/m')
    # plt.show()
    
    plt.subplot(1,2,2)

    MD_location_x = list()
    MD_location_y = list()
    MD_c = list()
    MD_tmp = np.zeros([4*16*4])
    for i in MD[int(MDevent[ind]):int(MDevent[ind + 1]), :]:
        MD_tmp[int(i[1])] = i[2]
    MD_tmp = MD_tmp.reshape(4,16,4)

    for i in range(4):
        for j in range(16):
            MD_c.append(MD_tmp[i,j,0])
            MD_location_x.append(x[(i+1)*1000+j*10])
            MD_location_y.append(y[(i+1)*1000+j*10])

    
    # plt.figure(figsize=(20, 16))
    
    cmapMD = cm.cool
    
    plt.scatter(
        MD_location_x,
        MD_location_y,
        c=MD_c,
        cmap=cmapMD,
        # edgecolors='b',
        linewidths=0.8,
        s=100,
        # alpha=0.5
    )
    # plt.clim(-3000, 3000)
    plt.colorbar()

    plt.scatter(pri_core_x[ind], pri_core_y[ind], marker='X', s=100, linewidths=1)
    arrowlength = 60
    plt.arrow(pri_core_x[ind] - arrowlength * np.cos(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              pri_core_y[ind] - arrowlength * np.sin(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              (arrowlength - 5) * np.cos(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              (arrowlength - 5) * np.sin(pri_phi[ind] + np.deg2rad(90 - 17.97)),
              head_width=3,
              fc='b',
              ec='b')
    plt.xlim(-150,150)
    plt.ylim(-150,150)
    plt.xlabel('x/m')
    plt.ylabel('y/m')

    plt.text(50,
    140,
    f'Energy: {pri_e[ind]/1000:5.2f} TeV\n'
    f'id: {pri_id[ind]}\n'
    f'theta: {np.rad2deg(pri_theta[ind]):5.2f} degree\n'
    f'phi: {np.rad2deg(pri_phi[ind]+ np.deg2rad(90 - 17.97)):5.2f} degree\n'
    f'ne: {pri_ne[ind]}',
    fontsize=20,
    verticalalignment="top",
    horizontalalignment="left"
)

    # print('theta', np.rad2deg(pri_theta[ind]), 'phi', np.rad2deg(pri_phi[ind]) + 90 - 17.97)
    plt.savefig(f'/home/hky/github/Identification_CR/P00_readMCdata/fig/{pri_id[ind]}_{int(pri_e[ind]/1000)}_event{ind}.png')
    plt.close()
